# Introduction to Natural Language Processing 01 

## Lab 03

## Features (6 points)

For every given text, we want to generate a vector with the features seen in class.

**(6 points)** Code the following features:
* 1 if "no" appears in the document, 0 otherwise.
* The count of first and second pronouns in the document.
* 1 if "!" is in the document, 0 otherwise.
* Log(word count in the document).
* Number of words in the document which are in the positive lexicon.
* Number of words in the document which are in the negative lexicon.
* **\[Bonus\]** Add another feature of your choice.

For positive and negative lexicons, you can use the resources provided by [VADER sentiment](https://github.com/cjhutto/vaderSentiment). Look for the `vader_lexicon.txt` file and consider positive word if they score above a certain threshold (for example 1) and negative word if they score below a certain threshold (for example -1). Feel free to use another lexicon if you find one, but make sure you document your choice.


### Tips

* Don't forget to use a similar pre-treatment as the one you used for the previous lab.
* Beware that words in the VADER dictionary are not lemmatized or stemmed. Do not use these pretreatments here.
* When checking for occurences of "no" or pronouns, split the text into token. Just using `"no" in text` would return true if the word "notable" is in your text.


In [4]:
import datasets as ds
import numpy as np

splits = ds.get_dataset_split_names('imdb')
train_ds = ds.load_dataset('imdb', split=splits[0])
test_ds = ds.load_dataset('imdb', split=splits[1])

Found cached dataset imdb (/Users/rb2/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Found cached dataset imdb (/Users/rb2/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


In [44]:
# For each given text, create a vector with the following features:

# if "no" appears in the document, 0 otherwise.
# The count of first pronouns (i , we) in the document.
# The count of second pronoun (you) in the document.
# 1 if "!" is in the document, 0 otherwise.
# Log(word count in the document).
# Number of positive words in the document.
# Number of negative words in the document.

# When checking for occurences of "no" or pronouns, split the text into token. Just using `"no" in text` would return true if the word "notable" is in your text.

#import `vader_lexicon.txt` from https://github.com/cjhutto/vaderSentiment and display it.
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

def get_features(text):
    text = text.lower()
    features = []
    positive = 0
    negative = 0
    features.append(1 if "no" in text.split() else 0)
    features.append(text.split().count("i") + text.split().count("we"))
    features.append(text.split().count("you"))
    features.append(1 if "!" in text else 0)
    features.append(np.log(len(text.split())))
    for word in text.split():
        if sid.lexicon.get(word, 0) < -1:
            negative += 1
        elif sid.lexicon.get(word, 0) > 1:
            positive += 1
    features.append(positive)
    features.append(negative)

    return features


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/rb2/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [57]:
get_features("I am You We  not happy NO !")

[1, 2, 1, 1, 2.0794415416798357, 1, 1]